# Imports & Setup

In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install nltk

In [2]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from operator import add

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  8 23:52 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'project_bucket_sy' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)

In [8]:
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name, prefix = 'wiki_files/')
for b in blobs:
    if b.name != 'wiki_files/graphframes.sh' and b.name != 'wiki_files/' :
        paths.append(full_path+b.name)

In [9]:
parquetFile = spark.read.parquet(*paths)

In [10]:
# Count number of wiki pages
parquetFile.count()


6348910

# Building an ID : Title dict

In [11]:
def build_doc_id_to_title_dict():
    return dict(parquetFile.select("id", "title").rdd.collect())

doc_id_to_title_dic = build_doc_id_to_title_dict()

In [12]:
name = "doc_id_to_title_dict"
with open(f"{name}.pkl", "wb") as f:
    pickle.dump(doc_id_to_title_dic, f)

client = storage.Client()
bucket = client.bucket(bucket_name)
blob_posting_locs = bucket.blob(f"title_id_dict/{name}.pkl")
blob_posting_locs.upload_from_filename(f"{name}.pkl")